# Improving Feature Engineering
*Anders Poirel*

In the previous notebook I realized that one-hot-encoding resulted in a feature matrix that was way to large to be processed in memory, so a new approach was needed. I either need to make my dataloaded batch the dataset OR come up with a more compact encoding

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

## Improved feature encoding

In [2]:
TRAIN_PATH = '../data/raw/train.csv'
TEST_PATH = '../data/raw/test.csv' 

In [3]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

In [4]:
X_train = train.iloc[:, 1:12]

In [5]:
X_train['City'] = train['City']

In [ ]:
X_train.columns.values

### Street Names

In [ ]:
names = X_train['EntryStreetName'].unique()

Let's take a look at how the street names to see if there is some clever feature engineering to do

In [ ]:
for name in names:
    print(name)

In [ ]:
X_train.groupby('EntryStreetName')['EntryStreetName'].count()

We have a lot of different entries so we might want to aggregate by street type (we found these types by looking manually throught the data):

In [9]:
street_types = [
    'Boulevard', 'Street', 'Avenue', 'Drive', 'Parkway', 'Road', 'Place', 'Way', 
    'Highway', 'Bridge', 'Tunnel', 'Terrace', 'Square',
    'Connector', 'Lane', 'Broadway', 'Wharf', 'Court', 'Circle',
]

# connector should be tested for before street
# street sometimes spelt st

In [ ]:
X_train.head()

In [11]:
def encode_type(street_name):
    street_types = [
        'Boulevard', 'Street', 'Avenue', 'Drive', 'Parkway', 'Road', 'Place', 'Way', 
        'Highway', 'Bridge', 'Tunnel', 'Terrace', 'Square',
        'Connector', 'Lane', 'Broadway', 'Wharf', 'Court', 'Circle',
        ]
    
    if pd.isna(street_name):
        return 'Not reported'
    
    # special cases to deal with redundant street names
    elif 'St' in street_name:
        return 'Street'
    elif 'Pkway' in street_name:
        return 'Parkway'
    
    else:
        for street_type in street_types:
            if street_type in street_name:
                return street_type
            
        return 'Other'

In [12]:
X_train['EntryStreetType'] = X_train['EntryStreetName'].apply(encode_type)
X_train['ExitStreetType'] = X_train['ExitStreetName'].apply(encode_type)

That being said, there are only 1700 different street types, so we might find it interesting to try fitting a model with one-hot-encoding on these, given how some of them appear several thousand times (and so we have a large sample for each category)

### Directions

Credits to D C Achaira's great Kaggle kernel for ideas on building more informative encodings for `EntryHeading` and `ExitHeading`:
[Feature Engineering and LightGBM](https://www.kaggle.com/dcaichara/feature-engineering-and-lightgbm)
I adapt his method directly here, computing the cardinal entry and exit directions, 
as well as the difference betweent the two.
This encoding is more informative as to similar directions will have more similar encodings.

In [13]:
def encode_direction(direction):
    encodings = {
        'N': 0,
        'NE': 0.25,
        'E':  0.5,
        'SE': 0.75,
        'S': 1,
        'SW': 1.25,
        'W': 1.5,
        'NW': 1.75,
    }
    return encodings[direction]

In [14]:
X_train['EntryHeading'] = X_train['EntryHeading'].apply(encode_direction)


In [15]:
X_train['ExitHeading'] = X_train['ExitHeading'].apply(encode_direction)

In [16]:
X_train['EntryExitDiff'] = X_train['ExitHeading'] - X_train['EntryHeading']

In [17]:
X_train.drop(['EntryStreetName', 'ExitStreetName', 'Path',
              'IntersectionId'], axis = 1, inplace = True)

In [18]:
X_train.head()

,Latitude,Longitude,EntryHeading,ExitHeading,Hour,Weekend,Month,City,EntryStreetType,ExitStreetType,EntryExitDiff
0,33.79166,-84.43003,1.75,1.75,0,0,6,Atlanta,Boulevard,Boulevard,0.0
1,33.79166,-84.43003,0.75,0.75,0,0,6,Atlanta,Boulevard,Boulevard,0.0
2,33.79166,-84.43003,1.75,1.75,1,0,6,Atlanta,Boulevard,Boulevard,0.0
3,33.79166,-84.43003,0.75,0.75,1,0,6,Atlanta,Boulevard,Boulevard,0.0
4,33.79166,-84.43003,1.75,1.75,2,0,6,Atlanta,Boulevard,Boulevard,0.0


I will deal will encodings for intersectionID and more advanced feature engineering once I get a baseline model up and working

### Tensorflow boilerplate for preprocessing
I'm still learning my way around tensorflow's `feature_columns` and while a lot of this could probably have been done there it's easier for me to do in pandas then convert the results using some boilerplate from `feature_columns` for OHE. I'm sure there's a lot of power I'm not leveraging there though. 


This boilerplate is similar to that in the previous notebook so I won't comment on it here

In [19]:
y_train_1 = train['TotalTimeStopped_p20']
y_train_2 = train['TotalTimeStopped_p50']
y_train_3 = train['TotalTimeStopped_p80']

In [22]:
CATEGORICAL_COLUMNS = ['Hour', 'Weekend', 'Month', 'City', 'EntryStreetType', 'ExitStreetType']

NUMERIC_COLUMNS = ['Latitude', 'Longitude', 'EntryHeading', 'ExitHeading', 'EntryExitDiff']

In [20]:
def make_input(X, y, n_epochs = None, shuffle = False):
    def input_f():
        dataset = tf.data.Dataset.from_tensor_slices((dict(X), y)) 
        if shuffle:
            dataset = dataset.shuffle(len(y))
        dataset = dataset.repeat(n_epochs)
        dataset = dataset.batch(len(y))
        return dataset
    return input_f

In [23]:
def one_hot_cat_column(feature_name, vocab):
    return tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocab))

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = X_train[feature_name].unique()
    feature_columns.append(one_hot_cat_column(feature_name, vocabulary))
    
for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                                            dtype = tf.float32))

#### Verification
Let's verify that this all works using the baseline `LinearRegressor` Estimator

In [ ]:
model_linear = tf.estimator.LinearRegressor(feature_columns)
input_fn = make_input(X_train, y_train_1)
model_linear.train(input_fn, max_steps = 20)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Anders\\AppData\\Local\\Temp\\tmpiufzb3sa', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001BB067A7108>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for

In [ ]:
metrics = model_linear.evaluate(input_fn)

## Modeling

In [ ]:
import hyperopt as hp
from tensorflow.estimator import BoostedTreesRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

We define the objective function for `hyperopt` to optimize, similar to the previous notebook:

In [ ]:
def rmse_cv_score(X, y, n_folds, kwargs, feature_columns):
    scores = []
    model = BoostedTreesRegressor(feature_columns, n_batches_per_layer = 1, **kwargs)

    for train_index, val_index in KFold(n_splits = 5).split(X):
        
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        train_input = make_input(X_train, y_train)
        val_input = make_input(X_val, y_val)
        
        model.train(train_input, max_steps = 1)
        
        scores.append(tf.math.sqrt(model.evaluate(val_input)))
        
    return np.mean(scores)

Define the search space for hyperopt. I decided on these paramaters by generalizing from my experience in tuning `xgboost`

In [ ]:
param_space = {

}